In [86]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# lscs members
lscs = pd.read_csv(
    "lscs_members.csv",  # 👈 disable quoting
    on_bad_lines="warn"      # skip/warn bad rows
)

lscs.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/tmp/ipython-input-3891465793.py:10: ParserWarning: Skipping line 743: expected 13 fields, saw 15

  lscs = pd.read_csv(


,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link
0,11825782,"Bernaldez, Leo Mikhail B.",NaN,leo_bernaldez@dlsu.edu.ph,NaN,MEM,MEM,GCOE,BS - Chemical Engineering,NaN,NaN,09052898843,https://www.facebook.com/beltran.bernaldez
1,11941820,"De Jesus, Matthew Joseph M.",NaN,matthew.dejesus@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies (CCS),BS - Computer Science Major in Network and Inf...,NaN,"Gaming, Editing",0917 618 8375,https://www.facebook.com/matti42/
2,12033391,"Palencia, Ralph Dwanne V",NaN,ralph_palencia@dlsu.edu.ph,NaN,MEM,MEM,SOE,BS Applied Industrial Economics,NaN,"Tech, games",09567349274,fb.com/dwanne.palencia
3,12038660,"Fong, Hannah Regine C.",NaN,hannah_regine_fong@dlsu.edu.ph,NaN,MEM,MEM,CCS,BSMS - Computer Science,NaN,NaN,09177012643,fb.com/fong.hannah
4,12073067,"de Ramos, Ghrazielle Rei A.",NaN,ghrazielle_deramos@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS Computer Science Major in Software Technology,NaN,"Basketball, Sketching, Coding, Volunteering",0933 862 0716,https://web.facebook.com/ghrazielle/


In [87]:
# houses
houses = {
    "Red": {
        "House Name": "House of Jasper",
        "Gem/Crystal": "Jasper",
        "Signature Color/Look": "earthy red/brown-red"
    },
    "Orange": {
        "House Name": "House of Carnelian",
        "Gem/Crystal": "Carnelian",
        "Signature Color/Look": "fiery orange"
    },
    "Yellow": {
        "House Name": "House of Citrine",
        "Gem/Crystal": "Citrine",
        "Signature Color/Look": "golden yellow"
    },
    "Green": {
        "House Name": "House of Aventurine",
        "Gem/Crystal": "Aventurine",
        "Signature Color/Look": "shimmering green"
    },
    "Light Blue": {
        "House Name": "House of Amazonite",
        "Gem/Crystal": "Amazonite",
        "Signature Color/Look": "turquoise blue-green"
    },
    "Indigo/Dark Blue": {
        "House Name": "House of Lapis Lazuli",
        "Gem/Crystal": "Lapis Lazuli",
        "Signature Color/Look": "deep royal blue"
    },
    "Purple": {
        "House Name": "House of Amethyst",
        "Gem/Crystal": "Amethyst",
        "Signature Color/Look": "violet to purple"
    },
    "White": {
        "House Name": "House of Moonstone",
        "Gem/Crystal": "Moonstone",
        "Signature Color/Look": "milky white with rainbow sheen"
    },
    "Black": {
        "House Name": "House of Obsidian",
        "Gem/Crystal": "Obsidian / Onyx",
        "Signature Color/Look": "jet black (volcanic glass)"
    },

    "Pink/Brown": {
        "House Name": "House of Rose Quartz",
        "Gem/Crystal": "Serpentine",
        "Signature Color/Look": "pink"
    }
}

house_df = pd.DataFrame.from_dict(houses, orient="index")
house_df.index.name = "Color Category"
house_df.to_csv("houses.csv", index=True)
house_df

,House Name,Gem/Crystal,Signature Color/Look
Color Category,,,
Red,House of Jasper,Jasper,earthy red/brown-red
Orange,House of Carnelian,Carnelian,fiery orange
Yellow,House of Citrine,Citrine,golden yellow
Green,House of Aventurine,Aventurine,shimmering green
Light Blue,House of Amazonite,Amazonite,turquoise blue-green
Indigo/Dark Blue,House of Lapis Lazuli,Lapis Lazuli,deep royal blue
Purple,House of Amethyst,Amethyst,violet to purple
White,House of Moonstone,Moonstone,milky white with rainbow sheen
Black,House of Obsidian,Obsidian / Onyx,jet black (volcanic glass)


In [88]:
specific = lscs[lscs["full_name"].str.contains("Rohann", case=False)]
specific

,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link
254,12343765,"Dizon, Rohann Gabriel D.",NaN,rohann_dizon@dlsu.edu.ph,NaN,VP,RND,College of Computer Studies,BS - CS ST,NaN,Volleyball,0967 087 1244,https://www.facebook.com/roedzn


In [89]:
# Clean your "interests" column
lscs["cleaned_interests"] = (
    lscs["interests"]
    .astype(str)
    .replace(r"[&/;\n|*+-]", ",", regex=True)  # added + and -
    .str.replace(r",+", ",", regex=True)        # collapse multiple commas
    .str.strip(", ")                            # strip leading/trailing commas/spaces
)

lscs['cleaned_interests'].sort_values()

,cleaned_interests
746,1. Coding (I worked on a platform game with tw...
789,3D Modeling and Video Editing
5,?
708,"ACADEMIC,Math, Science, History, Philosophy,MU..."
777,"Anime, Deepwoken, Overwatch, Pokémon, Genshin ..."
...,...
37,what if ayaw ko
56,writing
802,"writing, drawing, tech, games, music"
456,"writing, graphic design, walking"


In [93]:
def extract_keywords(text):
    if not text:
        return []
    words = re.findall(r'\b[a-zA-Z]{3,}\b', text.lower())
    return [w for w in words if w not in stop_words]

lscs["keywords_list"] = lscs["cleaned_interests"].apply(extract_keywords)
lscs


,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link,cleaned_interests,keywords_list
0,11825782,"Bernaldez, Leo Mikhail B.",NaN,leo_bernaldez@dlsu.edu.ph,NaN,MEM,MEM,GCOE,BS - Chemical Engineering,NaN,NaN,09052898843,https://www.facebook.com/beltran.bernaldez,nan,[nan]
1,11941820,"De Jesus, Matthew Joseph M.",NaN,matthew.dejesus@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies (CCS),BS - Computer Science Major in Network and Inf...,NaN,"Gaming, Editing",0917 618 8375,https://www.facebook.com/matti42/,"Gaming, Editing","[gaming, editing]"
2,12033391,"Palencia, Ralph Dwanne V",NaN,ralph_palencia@dlsu.edu.ph,NaN,MEM,MEM,SOE,BS Applied Industrial Economics,NaN,"Tech, games",09567349274,fb.com/dwanne.palencia,"Tech, games","[tech, games]"
3,12038660,"Fong, Hannah Regine C.",NaN,hannah_regine_fong@dlsu.edu.ph,NaN,MEM,MEM,CCS,BSMS - Computer Science,NaN,NaN,09177012643,fb.com/fong.hannah,nan,[nan]
4,12073067,"de Ramos, Ghrazielle Rei A.",NaN,ghrazielle_deramos@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS Computer Science Major in Software Technology,NaN,"Basketball, Sketching, Coding, Volunteering",0933 862 0716,https://web.facebook.com/ghrazielle/,"Basketball, Sketching, Coding, Volunteering","[basketball, sketching, coding, volunteering]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,12546615,"Angeles, Immanuel Zion B.",NaN,Immanuel_angeles@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - CSST18,NaN,"Coding, working out, basketball, guitar, running",0921 588 4310,https://www.facebook.com/share/18nUrcwsUy/,"Coding, working out, basketball, guitar, running","[coding, working, basketball, guitar, running]"
849,12547891,"Bai, Wilfred Hans T.",NaN,wilfred_bai@dlsu.edu.ph,NaN,MEM,MEM,CLA,AB Psychology / BS Marketing Management,NaN,"Listening to music, playing badminton",0917 865 5565,https://www.facebook.com/wilfred.bai,"Listening to music, playing badminton","[listening, music, playing, badminton]"
850,12548219,"Chua, Rayne Hyden S.",NaN,rayne_hyden_s_chua@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies,BS - Computer Science in Software Technology,NaN,Playing games and watching shows.,0947 606 6403,https://www.facebook.com/rayne.chua.2024,Playing games and watching shows.,"[playing, games, watching, shows]"
851,12587613,"Soque, Michael Paulo, L",NaN,michael_soque@dlsu.edu.ph,NaN,MEM,MEM,CCS,Master in Information Security,NaN,NaN,0961 150 1515,https://www.facebook.com/BoyWonderOfTheFutureP...,nan,[nan]


In [97]:
lscs["keywords"] = lscs["keywords_list"].apply(lambda x: ", ".join(x) if x else "")
lscs['keywords_list']


,keywords_list
0,[nan]
1,"[gaming, editing]"
2,"[tech, games]"
3,[nan]
4,"[basketball, sketching, coding, volunteering]"
...,...
848,"[coding, working, basketball, guitar, running]"
849,"[listening, music, playing, badminton]"
850,"[playing, games, watching, shows]"
851,[nan]


In [98]:
# Only vectorize non-empty rows
non_empty = lscs["keywords"] != ""
vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(", "))
X = vectorizer.fit_transform(lscs.loc[non_empty, "keywords"])


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [100]:
kmeans = KMeans(n_clusters=10, random_state=42)
lscs.loc[non_empty, "cluster"] = kmeans.fit_predict(X)

# Fill empty clusters with -1
lscs["cluster"] = lscs["cluster"].fillna(-1).astype(int)
lscs.head()

lscs.to_csv('assigned_houses.csv')